In [1]:
import pandas as pd
import numpy as np
import random
import operator
import math
import time
from pandas import DataFrame, read_csv
from numpy import linalg as LA
import skfuzzy as fuzz
import copy
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import calendar
import datetime
from datetime import datetime
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [2]:
#read the data and the preprocess version of the data.
carData = pd.read_csv('fina')
carData.shape


OSError: File b'dataWithDate.csv' does not exist

In [3]:
carData.head()

,Make,AccidentArea,Sex,MaritalStatus,Fault,PolicyType,VehiclePrice,FraudFound,RepNumber,Deductible,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,AccidentDate,ClaimDate,DayDiff
0,Honda,Urban,Female,Single,Policy Holder,Sport - Liability,"more than 69,000",0,12,300,...,26 to 30,No,No,External,none,1 year,3 to 4,1994-12-28,1993-12-28,0
1,Honda,Urban,Male,Single,Policy Holder,Sport - Collision,"more than 69,000",0,15,400,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994-01-12,1994-01-17,5
2,Honda,Urban,Male,Married,Policy Holder,Sport - Collision,"more than 69,000",0,7,400,...,41 to 50,No,No,External,none,no change,1 vehicle,1994-10-28,1994-11-10,13
3,Toyota,Rural,Male,Married,Third Party,Sedan - Liability,"20,000 to 29,000",0,4,400,...,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994-06-11,1994-07-01,20
4,Honda,Urban,Female,Single,Third Party,Sport - Collision,"more than 69,000",0,3,400,...,31 to 35,No,No,External,none,no change,1 vehicle,1994-01-24,1994-02-08,15


In [8]:
carData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15419 entries, 0 to 15418
Data columns (total 25 columns):
Make                    15419 non-null object
AccidentArea            15419 non-null object
Sex                     15419 non-null object
MaritalStatus           15419 non-null object
Fault                   15419 non-null object
PolicyType              15419 non-null object
VehiclePrice            15419 non-null object
FraudFound              15419 non-null int64
RepNumber               15419 non-null int64
Deductible              15419 non-null int64
DriverRating            15419 non-null int64
Days:Policy-Accident    15419 non-null object
Days:Policy-Claim       15419 non-null object
PastNumberOfClaims      15419 non-null object
AgeOfVehicle            15419 non-null object
AgeOfPolicyHolder       15419 non-null object
PoliceReportFiled       15419 non-null object
WitnessPresent          15419 non-null object
AgentType               15419 non-null object
NumberOfSuppliments  

In [9]:
#copy the label of fraud
carLabel = carData['FraudFound']

#drop the following attributes
carData.drop(['FraudFound'],inplace=True,axis=1)
carData.drop(['AccidentDate'],inplace=True,axis=1)
carData.drop(['ClaimDate'],inplace=True,axis=1)



In [10]:
carData.head()

,Make,AccidentArea,Sex,MaritalStatus,Fault,PolicyType,VehiclePrice,RepNumber,Deductible,DriverRating,...,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,DayDiff
0,Honda,Urban,Female,Single,Policy Holder,Sport - Liability,"more than 69,000",12,300,1,...,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,0
1,Honda,Urban,Male,Single,Policy Holder,Sport - Collision,"more than 69,000",15,400,4,...,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,5
2,Honda,Urban,Male,Married,Policy Holder,Sport - Collision,"more than 69,000",7,400,3,...,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,13
3,Toyota,Rural,Male,Married,Third Party,Sedan - Liability,"20,000 to 29,000",4,400,2,...,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,20
4,Honda,Urban,Female,Single,Third Party,Sport - Collision,"more than 69,000",3,400,1,...,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,15


In [142]:
#copy all the objects related attribute to the one dataframe
carData_Object = carData.select_dtypes(include=['object']).copy()

#from original drop all the objects attribute.
carData.drop(carData.select_dtypes(['object']), inplace=True, axis=1)


carData.head()

,RepNumber,Deductible,DriverRating,DayDiff
0,12,300,1,0
1,15,400,4,5
2,7,400,3,13
3,4,400,2,20
4,3,400,1,15


In [143]:
# converting categorical attributes to the binary Encodings:
# binary encodings: First assigns the binary value and converts it into binary representation.
import category_encoders as ce

encoderMake = ce.BinaryEncoder(cols=['Make'])
encoderMarital = ce.BinaryEncoder(cols=['MaritalStatus'])
encoderPolicyType = ce.BinaryEncoder(cols=['PolicyType'])
encoderVehiclePrice = ce.BinaryEncoder(cols=['VehiclePrice'])
encoderDayPA = ce.BinaryEncoder(cols=['Days:Policy-Accident'])
encoderDayPC = ce.BinaryEncoder(cols=['Days:Policy-Claim'])
encoderPNC = ce.BinaryEncoder(cols=['PastNumberOfClaims'])
encoderAgeOfVehicle = ce.BinaryEncoder(cols=['AgeOfVehicle'])
encoderAgeOfPolicyHolder = ce.BinaryEncoder(cols=['AgeOfPolicyHolder'])
encoderNoOfSuppliments = ce.BinaryEncoder(cols=['NumberOfSuppliments'])
encoderAddressChangeClaims = ce.BinaryEncoder(cols=['AddressChange-Claim'])
encoderNumberOfCars = ce.BinaryEncoder(cols=['NumberOfCars'])


#now transform to binary value
df_binary = encoderNumberOfCars.fit_transform(carData_Object)
df_binary = encoderAddressChangeClaims.fit_transform(df_binary)
df_binary = encoderNoOfSuppliments.fit_transform(df_binary)
df_binary = encoderAgeOfPolicyHolder.fit_transform(df_binary)
df_binary = encoderAgeOfVehicle.fit_transform(df_binary)
df_binary = encoderPNC.fit_transform(df_binary)
df_binary = encoderDayPC.fit_transform(df_binary)
df_binary = encoderDayPA.fit_transform(df_binary)
df_binary = encoderVehiclePrice.fit_transform(df_binary)
df_binary = encoderPolicyType.fit_transform(df_binary)
df_binary = encoderMarital.fit_transform(df_binary)
df_binary = encoderMake.fit_transform(df_binary)

In [144]:
df_binary.head()

,Make_0,Make_1,Make_2,Make_3,Make_4,MaritalStatus_0,MaritalStatus_1,PolicyType_0,PolicyType_1,PolicyType_2,...,AddressChange-Claim_2,NumberOfCars_0,NumberOfCars_1,NumberOfCars_2,AccidentArea,Sex,Fault,PoliceReportFiled,WitnessPresent,AgentType
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,Urban,Female,Policy Holder,No,No,External
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,Urban,Male,Policy Holder,Yes,No,External
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,1,Urban,Male,Policy Holder,No,No,External
3,0,0,0,0,1,0,1,0,0,1,...,1,0,0,1,Rural,Male,Third Party,Yes,No,External
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,Urban,Female,Third Party,No,No,External


In [145]:
#convert the remaining attributes to one-hot encodings
df_binary = pd.get_dummies(df_binary,columns=['AccidentArea'],prefix=['AccidentArea'])
df_binary = pd.get_dummies(df_binary,columns=['Sex'],prefix=['Sex'])
df_binary = pd.get_dummies(df_binary,columns=['Fault'],prefix=['Fault'])
df_binary = pd.get_dummies(df_binary,columns=['PoliceReportFiled'],prefix=['PoliceReportFiled'])
df_binary = pd.get_dummies(df_binary,columns=['WitnessPresent'],prefix=['WitnessPresent'])
df_binary = pd.get_dummies(df_binary,columns=['AgentType'],prefix=['AgentType'])


In [146]:
df_binary.head()

,Make_0,Make_1,Make_2,Make_3,Make_4,MaritalStatus_0,MaritalStatus_1,PolicyType_0,PolicyType_1,PolicyType_2,...,Sex_Female,Sex_Male,Fault_Policy Holder,Fault_Third Party,PoliceReportFiled_No,PoliceReportFiled_Yes,WitnessPresent_No,WitnessPresent_Yes,AgentType_External,AgentType_Internal
0,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
2,0,0,0,0,0,0,1,0,0,0,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0,0,0,0,1,0,1,0,0,1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0


In [147]:
#convert all the float to int
rural= df_binary['AccidentArea_Rural'].astype('int64')
urban= df_binary['AccidentArea_Urban'].astype('int64')
female= df_binary['Sex_Female'].astype('int64')
male= df_binary['Sex_Male'].astype('int64')
policyHolder= df_binary['Fault_Policy Holder'].astype('int64')
thirdParty= df_binary['Fault_Third Party'].astype('int64')
policeReport_No= df_binary['PoliceReportFiled_No'].astype('int64')
policeReport_Yes= df_binary['PoliceReportFiled_Yes'].astype('int64')
witness_No= df_binary['WitnessPresent_No'].astype('int64')
witness_Yes= df_binary['WitnessPresent_Yes'].astype('int64')
external= df_binary['AgentType_External'].astype('int64')
internal= df_binary['AgentType_Internal'].astype('int64')

#convert Series to dataFrame
rural= rural.to_frame()
urban= urban.to_frame()
female= female.to_frame()
male= male.to_frame()
policyHolder= policyHolder.to_frame()
thirdParty= thirdParty.to_frame()
policeReport_No= policeReport_No.to_frame()
policeReport_Yes= policeReport_Yes.to_frame()
witness_No= witness_No.to_frame()
witness_Yes= witness_Yes.to_frame()
external= external.to_frame()
internal= internal.to_frame()

#drop the float type attributes
df_binary.drop(['AccidentArea_Rural'],inplace=True,axis=1)
df_binary.drop(['AccidentArea_Urban'],inplace=True,axis=1)
df_binary.drop(['Sex_Female'],inplace=True,axis=1)
df_binary.drop(['Sex_Male'],inplace=True,axis=1)
df_binary.drop(['Fault_Policy Holder'],inplace=True,axis=1)
df_binary.drop(['Fault_Third Party'],inplace=True,axis=1)
df_binary.drop(['PoliceReportFiled_No'],inplace=True,axis=1)
df_binary.drop(['PoliceReportFiled_Yes'],inplace=True,axis=1)
df_binary.drop(['WitnessPresent_No'],inplace=True,axis=1)
df_binary.drop(['WitnessPresent_Yes'],inplace=True,axis=1)
df_binary.drop(['AgentType_External'],inplace=True,axis=1)
df_binary.drop(['AgentType_Internal'],inplace=True,axis=1)


df_binary.head()


,Make_0,Make_1,Make_2,Make_3,Make_4,MaritalStatus_0,MaritalStatus_1,PolicyType_0,PolicyType_1,PolicyType_2,...,AgeOfPolicyHolder_2,AgeOfPolicyHolder_3,NumberOfSuppliments_0,NumberOfSuppliments_1,AddressChange-Claim_0,AddressChange-Claim_1,AddressChange-Claim_2,NumberOfCars_0,NumberOfCars_1,NumberOfCars_2
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1
3,0,0,0,0,1,0,1,0,0,1,...,1,1,0,1,0,0,1,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [148]:
# add the converted int attributes to the dataframe
df_binary['AccidentArea_Rural'] = rural['AccidentArea_Rural']
df_binary['AccidentArea_Urban'] = urban['AccidentArea_Urban']
df_binary['Sex_Female'] = female['Sex_Female']
df_binary['Sex_Male'] = male['Sex_Male']
df_binary['Fault_Policy Holder'] = policyHolder['Fault_Policy Holder']
df_binary['Fault_Third Party'] = thirdParty['Fault_Third Party']
df_binary['PoliceReportFiled_No'] = policeReport_No['PoliceReportFiled_No']
df_binary['PoliceReportFiled_Yes'] = policeReport_Yes['PoliceReportFiled_Yes']
df_binary['WitnessPresent_No'] = witness_No['WitnessPresent_No']
df_binary['WitnessPresent_Yes'] = witness_Yes['WitnessPresent_Yes']
df_binary['AgentType_External'] = external['AgentType_External']
df_binary['AgentType_Internal'] = internal['AgentType_Internal']


#add the remaining attributes

df_binary['RepNumber'] = carData['RepNumber']
df_binary['Deductible'] = carData['Deductible']
df_binary['DriverRating'] = carData['DriverRating']
df_binary['DayDiff'] = carData['DayDiff']



df_binary.head()





,Make_0,Make_1,Make_2,Make_3,Make_4,MaritalStatus_0,MaritalStatus_1,PolicyType_0,PolicyType_1,PolicyType_2,...,PoliceReportFiled_No,PoliceReportFiled_Yes,WitnessPresent_No,WitnessPresent_Yes,AgentType_External,AgentType_Internal,RepNumber,Deductible,DriverRating,DayDiff
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,12,300,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,15,400,4,5
2,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,7,400,3,13
3,0,0,0,0,1,0,1,0,0,1,...,0,1,1,0,1,0,4,400,2,20
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,3,400,1,15


In [149]:
#save the binaryEncoding data 
df_binary.to_csv('finalPreprocessBinary.csv',sep=' ',encoding='utf-8')

In [ ]:
#aplly the fcm algorithm:'

